In [1]:
%load_ext autoreload

In [2]:
%matplotlib inline
%autoreload 2
import bisect
import copy 
import os 
from collections import deque, Counter
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import editdistance
import sys
import RNA
from typing import Dict, List, Tuple

# import path 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.sequence_utils import translate_one_hot_to_string,generate_random_mutant
from utils.sequence_utils import translate_string_to_one_hot, translate_one_hot_to_string
from models.Theoretical_models import *
from models.Noise_wrapper import *
from exploration_strategies.CE import *
from utils.landscape_utils import *
from models.RNA_landscapes import *
from models.Multi_dimensional_model import *

import tensorflow as tf
from tf_agents.drivers import dynamic_step_driver
from tf_agents.metrics import tf_metrics
from tf_agents.agents import tf_agent
from tf_agents.policies import random_tf_policy
from tf_agents.agents.ppo import ppo_policy, ppo_agent, ppo_utils
from tf_agents.environments import py_environment, tf_py_environment
from tf_agents.environments.utils import validate_py_environment
from tf_agents.drivers import dynamic_episode_driver
from tf_agents.networks import network, normal_projection_network
from tf_agents.networks import actor_distribution_network
from tf_agents.networks import actor_distribution_rnn_network
from tf_agents.networks import value_network
from tf_agents.networks import value_rnn_network
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.utils import common
from tf_agents.trajectories import time_step as ts
from tf_agents.specs import array_spec

Using TensorFlow backend.


In [3]:
RAA="UGCA" #alphabet
alphabet_len=len(RAA)
# TODO: UNDO THIS
# length=40
length=20
noise_alpha=1
generations = 10
experiment_batch_size = 1000
wt=generate_random_sequences(length,1,alphabet=RAA)[0]
landscape1=RNA_landscape(wt)
landscape2=RNA_landscape(wt)
noisy_landscape=Noise_wrapper(landscape1,
                              noise_alpha=noise_alpha,
                              always_costly=True)
initial_genotypes=list(set([wt]+[generate_random_mutant(wt,0.05,RAA) 
                                 for i in range(experiment_batch_size*10)]))[:experiment_batch_size]
noisy_landscape.reset()
noisy_landscape.measure_true_landscape(initial_genotypes)
noisy_landscape.natural_mode=False
noisy_landscape.local_mode=False
noisy_landscape.cost

1000

In [4]:
class FitnessLandscapeEnvironment(py_environment.PyEnvironment):
    # Based on this: https://www.mikulskibartosz.name/how-to-create-an-environment-for-a-tensorflow-agent/
    def __init__(self, alphabet, seq_len, landscape, max_episodes):
        self.alphabet = alphabet
        self.alphabet_len = len(self.alphabet)
        self.landscape = copy.deepcopy(landscape)
        self.seq_len = seq_len
        self._action_spec = array_spec.BoundedArraySpec(
            shape=(1, 2), dtype=np.float32, minimum=0, 
            maximum=1, name='action_x')
        self._observation_spec = array_spec.BoundedArraySpec(
            shape=(self.alphabet_len, self.seq_len), dtype=np.float32, minimum=0, 
            maximum=1, name='observation')
        self._time_step_spec = ts.time_step_spec(self._observation_spec)
        self._state = translate_string_to_one_hot(wt, self.alphabet)
        self._episode_ended = False
        self.ctr = 0
        self.max_episodes = max_episodes
        self.seen_sequences = {}
        
    def _reset(self):
        self.ctr = 0
        # TODO: UNDO THIS
#         self._state = translate_string_to_one_hot(wt, self.alphabet)
        self._state = translate_string_to_one_hot(generate_random_sequences(length,1,alphabet=RAA)[0], self.alphabet)
        self._episode_ended = False
        return ts.restart(np.array(self._state, dtype=np.float32))
    
    def time_step_spec(self):
        return self._time_step_spec 

    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec
    
    def get_state_string(self):
        return translate_one_hot_to_string(self._state, self.alphabet)
    
    def _step(self, action):
        if self.ctr < self.max_episodes:
            self.ctr += 1
            action_one_hot = np.zeros((self.alphabet_len, self.seq_len))
            if np.amax(action) > 1 or np.amin(action) < 0:
                
                self.seen_sequences = {}
                return ts.termination(np.array(self._state, dtype=np.float32), 0)
            x, y = action[0]
            x, y = int(self.alphabet_len*x), int(self.seq_len*y)
            action_one_hot[x, y] = 1
            assert self._state.sum() == self._state.shape[1]
            if self._state[x, y] == 1:
                self._episode_ended = True
                
                self.seen_sequences = {}
                return ts.termination(np.array(self._state, dtype=np.float32), 0)
            else:
#                 self._state = construct_mutant_from_sample(action_one_hot, self._state)
                new_stage = construct_mutant_from_sample(action_one_hot, self._state)
                state_string = translate_one_hot_to_string(new_stage, self.alphabet)
                try_iter = 0 
                while state_string in self.seen_sequences:
                    try_iter += 1
                    new_stage = construct_mutant_from_sample(action_one_hot, self._state)
                    state_string = translate_one_hot_to_string(new_stage, self.alphabet)
#                     print("www")
                    
                    if try_iter >= 1000:
                        self._state = translate_string_to_one_hot(wt, self.alphabet)
                        self.seen_sequences = {}
                        return ts.termination(np.array(self._state, dtype=np.float32), 0)
                                        
#                 if state_string in self.seen_sequences:
#                     self._state = translate_string_to_one_hot(wt, self.alphabet)
#                     self.seen_sequences = {}
#                     return ts.termination(np.array(self._state, dtype=np.float32), 0)

                assert state_string not in self.seen_sequences
                self._state=new_stage
                self.seen_sequences[state_string] = 1
                
                reward = self.landscape.get_fitness(state_string)
                assert self._state.sum() == self._state.shape[1]
                return ts.transition(np.array(self._state, dtype=np.float32), reward=reward)
        else:
            self._episode_ended = True
            assert self._state.sum() == self._state.shape[1]
            
            self.seen_sequences = {}
            return ts.termination(np.array(self._state, dtype=np.float32), 0)

In [5]:
#max_iter = experiment_batch_size * generations 
max_iter = 10 ** 6
fle2 = FitnessLandscapeEnvironment(RAA, length, landscape1, max_iter)
print("starting validating environment...")
validate_py_environment(fle2, episodes=2)
print("done validating environment.")
fle = FitnessLandscapeEnvironment(RAA, length, landscape2, max_iter)
tf_env = tf_py_environment.TFPyEnvironment(fle)

starting validating environment...
done validating environment.


In [6]:
# return
%autoreload 2
from PPO_utils import PPO_RL_Agent 

def BoostedEnvironment():
    return FitnessLandscapeEnvironment(RAA, length, landscape2, max_iter)

actor_fc_layers = (200, 100)
value_fc_layers = (200, 100)
env_load_fn = BoostedEnvironment
'''
actor_net = actor_distribution_rnn_network.ActorDistributionRnnNetwork(
    tf_env.observation_spec(),
    tf_env.action_spec(),
    input_fc_layer_params=actor_fc_layers,
    output_fc_layer_params=None
)
value_net = value_rnn_network.ValueRnnNetwork(
    tf_env.observation_spec(),
    input_fc_layer_params=value_fc_layers,
    output_fc_layer_params=None
)
'''
actor_net = actor_distribution_network.ActorDistributionNetwork(
  tf_env.observation_spec(),
  tf_env.action_spec(),
  fc_layer_params=actor_fc_layers)
value_net = value_network.ValueNetwork(
  tf_env.observation_spec(), fc_layer_params=value_fc_layers)
num = 100000
ppo_rl_agent = PPO_RL_Agent(env_load_fn, actor_net, value_net, RAA, num_environment_steps=500000)
ppo_rl_agent.run_train()
batch = ppo_rl_agent.pick_action()

SyntaxError: 'return' outside function (<ipython-input-6-28837e2f79b0>, line 1)

In [7]:
PPO_dist = get_distance_fitness(wt, noisy_landscape)
plt.scatter([x[0] for x in PPO_dist], [x[1] for x in PPO_dist])